In [1]:
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import MaximalMarginalRelevance
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
import pandas as pd
from pymilvus import connections, utility, Collection, FieldSchema, CollectionSchema, DataType
from tqdm import tqdm
from transformers import pipeline           # Hugging Face转换器
import networkx as nx                       # 网络图形可视化
import matplotlib.pyplot as plt
import json


## 参数选择

In [2]:
# 加载需要查看的embedding
input_embeddings = np.load('emb_a15b45.npy')  # 文本的embedding

# 用于进行tm的文本选择
# 'full' 'text' 'abstract'
full_text_abstract = 'full'


In [3]:
# 设置文件路径
raw_csv_path ="../raw.csv"

# 读取CSV文件
def load_info():
    # 读取csv文件
    df = pd.read_csv(raw_csv_path)
    
    # 提取需要的列
    info = []
    for _, row in df.iterrows():
        abstract = row['abstract']
        try:
            year = int(row['pub_year'])
        except:
            year = np.nan
        
        # 提取其他需要的列
        pmid = row.get('pmid', None)
        title = row.get('title', None)
        mesh_terms = row.get('mesh_terms', None)
        text = row.get('text', None)
        
        # 只添加需要的列
        info.append([abstract, year, pmid, title, mesh_terms, text])
    
    # 创建DataFrame，只包含指定的列
    info = pd.DataFrame(info, columns=['abstract', 'year', 'pmid', 'title', 'mesh_terms', 'text'])
    return info

# 加载数据
data_full = load_info()

# 移除缺失摘要的条目
data_full = data_full.dropna(subset=['abstract'])


# 打印基本信息
print(f"数据总量: {len(data_full)}")
print(f"年份范围: {data_full['year'].min()} - {data_full['year'].max()}")

数据总量: 3000
年份范围: 2002 - 2025


In [4]:
# 提取mesh的处理

# 提取mesh_terms中的DescriptorName并拼接
def extract_mesh(row):
    mesh_str = ""
    try:
        mesh_list = json.loads(row['mesh_terms'])
        terms = [item["DescriptorName"] for item in mesh_list if "DescriptorName" in item]
        mesh_str = " ".join(terms)
    except:
        pass
    return mesh_str

# 添加提取后的mesh_terms并与abstract拼接
data_full['mesh_extracted'] = data_full.apply(extract_mesh, axis=1)
data_full['full'] = data_full['title'] + " " + data_full['abstract'] + " " + data_full['mesh_extracted']
data_full['text'] = data_full['title'] + " " + data_full['abstract']

# text_field = 'full' if use_full_text else 'abstract'
docs = data_full[full_text_abstract].values  # 根据 full_text_abstract 变量选择使用全文或摘要
print(docs[:1])



['Anxiety and depression in healthcare workers 2 years after COVID-19 infection and scale validation. This study aims to assess the levels of anxiety and depression among healthcare workers two years post COVID-19 infection and to validate the reliability and validity of the PHQ-9 and GAD-7 scales in this population. This cross-sectional study was conducted in June 2024 using a simple random sampling approach to survey healthcare institution workers. A total of 1038 valid samples were collected, and anxiety and depression levels were assessed using the PHQ-9 and GAD-7 scales. Participants included healthcare workers such as doctors, nurses, administrative staff, and students. Data analysis included descriptive statistics, correlation analysis, univariate, and multivariate analyses to explore the effects of variables such as occupation and gender on anxiety and depression. Long COVID was reported in 50.8% of participants. Occupational categories significantly influenced anxiety and depr

In [5]:

# reduce dimensionality
umap_model = UMAP(
    n_neighbors = 15, n_components = 5, min_dist = 0.0, 
    metric = 'cosine', random_state = 34)
# cluster reduced embeddings
hdbscan_model = HDBSCAN(
    min_cluster_size = 30, min_samples = 10,  
    metric = 'euclidean', cluster_selection_method = 'eom', 
    prediction_data = True)

# tokenize topics
vectorizer_model = CountVectorizer(stop_words = "english")
# create topic representation
ctfidf_model = ClassTfidfTransformer()
# fine tune with relevance
representation_model = MaximalMarginalRelevance(diversity = 0.2)
# all steps together
topic_model = BERTopic(
  # embedding_model = embedding_model,    # Step 1 - Extract embeddings
  umap_model = umap_model,              # Step 2 - Reduce dimensionality
  hdbscan_model = hdbscan_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model = vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model = ctfidf_model,          # Step 5 - Extract topic words        
  calculate_probabilities = True,        
  verbose = True,
  representation_model = representation_model # Diversify topic words
)

In [7]:
# 训练模型
topics, probs = topic_model.fit_transform(docs,embeddings=input_embeddings)

# topic_model.fit_transform(docs, embeddings=embeddings)

# 获取聚类结果
topic_info = topic_model.get_topic_info()
topic_info



2025-04-28 14:07:23,710 - BERTopic - Reduced dimensionality
2025-04-28 14:07:23,884 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name,Representation,Representative_Docs
0,-1,997,-1_students_health_study_anxiety,"[students, health, study, anxiety, adult, medi...","[Depressive symptoms, college adjustment and p..."
1,0,274,0_intervention_group_anxiety_mindfulness,"[intervention, group, anxiety, mindfulness, tr...",[Effectiveness of a Web-Based Guided Self-help...
2,1,247,1_covid19_pandemic_health_mental,"[covid19, pandemic, health, mental, students, ...",[The impact of the COVID-19 epidemic on studen...
3,2,212,2_students_depression_prevalence_symptoms,"[students, depression, prevalence, symptoms, h...",[Depressive Symptoms Among Sarajevo University...
4,3,175,3_factor_validity_psychometric_scale,"[factor, validity, psychometric, scale, reliab...",[The psychometric properties of the Cognitive-...
5,4,155,4_burnout_residents_residency_resident,"[burnout, residents, residency, resident, trai...",[A One-Year Institutional View of Resident Phy...
6,5,146,5_students_clinical_nursing_learning,"[students, clinical, nursing, learning, simula...",[An integrated model for the effects of self-r...
7,6,131,6_patients_aged_depression_life,"[patients, aged, depression, life, cancer, fac...",[Implementation of a clinical pathway in breas...
8,7,101,7_health_use_numerical_mental,"[health, use, numerical, mental, data, college...",[Perceived stigma and mental health care seeki...
9,8,94,8_suicide_suicidal_ideation_symptoms,"[suicide, suicidal, ideation, symptoms, health...",[Sexual Victimization and Suicide Ideation: An...


In [8]:
# info about all docs assigned to topics
documents = topic_model.get_document_info(docs) 

In [9]:
# ====================================
# 模块5: 模型评估
# 计算主题一致性(TC)和主题多样性(TD)指标
# ====================================
# 将每个主题下的文档合并为一个长文档
documents_per_topic = documents.groupby(['Topic'], as_index = False).agg({'Document': ' '.join})
# 对文档进行预处理
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)
# 构建文档分析器
analyzer = topic_model.vectorizer_model.build_analyzer()
# 获取每个文档的标记
tokens = [analyzer(doc) for doc in cleaned_docs]
# 构建主题词列表(每个主题取前10个关键词的第一个元素)
bertopic_topics = [
    [topicwords[0] for topicwords in topic_model.get_topic(i)[:10]]
    for i in range(len(set(topics)) - 1)]
# 计算主题一致性(Topic Coherence)
TC = Coherence(texts = tokens, topk = 10, measure = 'c_v').score({'topics': bertopic_topics}) 
# 计算主题多样性(Topic Diversity)
TD = TopicDiversity().score({'topics': bertopic_topics})
# print('TC = ', TC, 'TD = ', TD)
# 输出保留4位小数的指标值
print(f'TC = {TC:.4f}, TD = {TD:.4f}')


TC = 0.5742, TD = 0.7056


In [33]:
# # ====================================
# # 模块6: 主题探索和分析
# # 对特定主题进行详细分析
# # ====================================
# # 选择一个感兴趣的主题进行分析
# MY_TOPIC = 5
# # 获取该主题的十个关键词
# topic_model.get_topic(MY_TOPIC) 
# # 获取分配给该主题的所有文档
# assigned_docs = documents[documents.Topic == MY_TOPIC] 
# # 打印该主题下所有文档的摘要
# for abstract in assigned_docs.Document:
#     print(abstract)
#     print('---')

# # 获取该主题的代表性文档
# representative_docs = topic_model.get_representative_docs(MY_TOPIC)
# # 获取代表性文档的引用信息
# data_full[data_full.abstract == representative_docs[1]].iloc[0].citation
    
# # 使用零样本分类器为主题分配标签
# classifier = pipeline("zero-shot-classification", model = "facebook/bart-large-mnli")
# # 将主题关键词组合成一个序列
# sequence_to_classify =  " ".join([word for word, _ in topic_model.get_topic(MY_TOPIC)])
# # 定义候选标签
# candidate_labels = ["Psychometrics of depression"]
# # 执行分类
# classifier(sequence_to_classify, candidate_labels)

In [34]:
# # ====================================
# # 模块7: 结果可视化
# # 使用多种方法可视化主题建模结果
# # ====================================
# # 1. 条形图 - 显示前六个主题的关键词
# barchart = topic_model.visualize_barchart(top_n_topics = 6, n_words = 5, width = 400, title = "")
# barchart.show()

# # 2. 主题间关系图 - 展示主题之间的距离和关系
# topic_model.visualize_topics().show()

# # 3. 热力图 - 显示主题之间的相似度矩阵
# topic_model.visualize_heatmap(n_clusters = 7)

# # 4. 层次聚类 - 展示主题的层次结构
# hierarchical_topics = topic_model.hierarchical_topics(docs)
# topic_model.visualize_hierarchy(hierarchical_topics = hierarchical_topics)

# # 5. 文档嵌入可视化 - 展示文档在空间中的分布
# embeddings = embedding_model.encode(docs, show_progress_bar = False)
# topic_model.visualize_documents(docs, embeddings = embeddings)   
# topic_model.visualize_hierarchical_documents(docs, hierarchical_topics, embeddings = embeddings) 
 
# # 6. 时间序列分析 - 展示主题随时间变化的趋势
# time = data_full.year.to_list() 
# topics_over_time = topic_model.topics_over_time(docs, time)
# topic_model.visualize_topics_over_time(topics_over_time, topics = [0, 4, 5, 20])

# # 7. 主题相关性搜索 - 查找与特定关键词相关的主题
# # 函数：绘制关键词与主题的相似度网络图
# def draw_simil(keyword):
#     # 找出与关键词最相关的3个主题及其相似度
#     topicsF, similarity = topic_model.find_topics(keyword, top_n = 3)
#     # 构建网络图的节点
#     G = nx.DiGraph()
#     G.add_node(-1)
#     G.add_node(0)
#     G.add_node(1)
#     G.add_node(2)
#     node_labels = dict(enumerate(topicsF))
#     node_labels[-1] = keyword
#     scale = (len(keyword) + 12) * 0.032
#     x_offset = 0.25 * scale
#     y_offset = -.5
#     # 设置节点位置
#     pos = {
#         -1: (0, 0),
#         0: (-x_offset * scale, y_offset),
#         1: (0, y_offset),
#         2: (x_offset * scale, y_offset)
#     }
#     # 绘制节点
#     nx.draw(
#         G, pos = pos, labels = node_labels, with_labels = True, node_shape = "s", 
#         bbox = dict(facecolor = "lightgreen", edgecolor = 'black', boxstyle = 'round,pad=0.5'))
    
#     # 构建边的图形(用于改变边的位置)
#     H = nx.DiGraph()
#     for i, sim_val in enumerate(similarity):
#         H.add_edge(i + 3, i, label = round(sim_val, 2)) # 3->0, 4->1, 5->2
#     y_offset = -.48
#     # 设置边位置
#     pos = {
#         0: (-x_offset * scale, y_offset),
#         1: (0, y_offset),
#         2: (x_offset * scale, y_offset),
#         3: (-x_offset * scale, 0),
#         4: (0, 0),
#         5: (x_offset * scale, 0)
#     }
#     edge_labels = nx.get_edge_attributes(H, "label")
#     # 绘制边和标签
#     nx.draw_networkx(H, labels = {}, node_color = "white", arrows = True, pos = pos)
#     nx.draw_networkx_edge_labels(H, pos, edge_labels)
#     # 设置图形边距
#     plt.xlim(-1, 1)
#     plt.ylim(-1, 1)
#     ax = plt.gca()
#     ax.margins(0.20)
#     plt.axis("off")
#     plt.show()

# # 分析两个关键词与主题的相关性
# draw_simil('internet addiction')
# draw_simil('burnout measures')
